# Marketplace for Sub-Urban Housing Complex (Indonesia)
This notebook models a digital marketplace for a neighborhood/sub-urban housing complex (~1000 houses) in Indonesia. The main currency is Indonesian Rupiah (IDR). Sellers can create digital stores, manage products, and offer various payment methods (Cash on Delivery, bank transfer, QRIS with QR upload). Sellers are responsible for delivery. Order status/notification features are planned for future implementation.

# Basic Marketplace Notebook Skeleton
This notebook provides a basic structure for a marketplace app, ready for further customization.

## 1. Import Required Libraries
Import Python libraries such as pandas for data handling and any other necessary modules.

In [ ]:
import pandas as pd
from typing import List

# Add any other necessary imports here

# Kelas dan Struktur Data (Terjemahan Lokal)
Berikut adalah definisi kelas dengan nama dan atribut yang dilokalkan untuk kemudahan pengembangan frontend dan backend.

In [ ]:
# Kelas Pengguna, Penjual, Pembeli, Produk, dan Toko dengan atribut lokal dan tambahan profil
import hashlib
from datetime import datetime
class Pengguna:
    def __init__(self, username: str, password: str, nama: str = None, no_hp: str = None, foto_profil: str = None):
        self.username = username
        self.password_hash = self.hash_password(password)
        self.nama = nama
        self.no_hp = no_hp
        self.foto_profil = foto_profil
        self.keranjang = []  # List of Produk IDs
    def hash_password(self, password):
        return hashlib.sha256(password.encode()).hexdigest()
    def cek_password(self, password):
        return self.password_hash == hashlib.sha256(password.encode()).hexdigest()

class Penjual(Pengguna):
    def __init__(self, username: str, password: str, nama: str = None, no_hp: str = None, foto_profil: str = None):
        super().__init__(username, password, nama, no_hp, foto_profil)
        self.toko_list = []  # List of Toko objects

class Pembeli(Pengguna):
    def __init__(self, username: str, password: str, nama: str = None, no_hp: str = None, alamat: str = None, foto_profil: str = None):
        super().__init__(username, password, nama, no_hp, foto_profil)
        self.alamat = alamat  # Alamat utama pembeli

class Produk:
    def __init__(self, id: int, nama: str, kategori: str, harga: int, deskripsi: str, url_gambar: str = None, status: str = "Tersedia"):
        self.id = id
        self.nama = nama
        self.kategori = kategori
        self.harga = harga  # Harga dalam Rupiah
        self.deskripsi = deskripsi
        self.url_gambar = url_gambar
        self.status = status  # "Tersedia" atau "Kosong"

class Toko:
    def __init__(self, id: int, penjual: 'Penjual', nama: str, deskripsi: str, logo: str = None, kontak: str = None, status: str = "Buka"):
        self.id = id
        self.penjual = penjual
        self.nama = nama
        self.deskripsi = deskripsi
        self.logo = logo  # URL logo toko
        self.kontak = kontak  # Nomor kontak (misal WhatsApp)
        self.status = status  # "Buka" atau "Tutup"
        self.produk_list = []  # List of Produk objects
        self.metode_pembayaran = []  # List of payment method strings
        self.qris_url = None  # Optional QRIS QR code image

# Sample sellers and stores
sellers = {}
stores = {}

seller1 = Seller("toko_andi", "password123")
sellers[seller1.username] = seller1
store1 = Store(1, seller1, "Toko Andi", "Sembako & kebutuhan harian")
store1.payment_methods = ["Cash on Delivery", "Bank Transfer", "QRIS"]
store1.qris_image_url = "https://example.com/qris-andi.png"  # Placeholder
seller1.stores.append(store1)
stores[store1.id] = store1

# Sample products for the store
product1 = Product(1, "Beras 5kg", "Sembako", 75000, "Beras premium 5kg", None)
product2 = Product(2, "Minyak Goreng 2L", "Sembako", 35000, "Minyak goreng kemasan 2L", None)
store1.products.extend([product1, product2])

# Convert to DataFrame for easier handling
def store_products_df(store: Store):
    return pd.DataFrame([vars(p) for p in store.products])

store_products_df(store1)

In [ ]:
products = [
    Product(1, "T-shirt", "Clothing", 19.99, "Comfortable cotton t-shirt"),
    Product(2, "Sneakers", "Footwear", 49.99, "Stylish running sneakers"),
    Product(3, "Backpack", "Accessories", 29.99, "Durable travel backpack"),
]

# Convert to DataFrame for easier handling
df_products = pd.DataFrame([vars(p) for p in products])
df_products

## 4. Implement Product Search Functionality
Write a function to search for products by name or category.

In [ ]:
def search_products(query: str, by: str = "name"):
    if by not in ["name", "category"]:
        raise ValueError("Search by 'name' or 'category' only.")
    return df_products[df_products[by].str.contains(query, case=False)]

# Example search
search_products("shirt")

users = {}

def register_user(username: str, password: str, is_seller: bool = False):
    if username in users:
        print("Username already exists.")
        return None
    if is_seller:
        user = Seller(username, password)
    else:
        user = User(username, password)
    users[username] = user
    print(f"User {username} registered as {'Seller' if is_seller else 'Buyer'}.")
    return user

def login_user(username: str, password: str):
    user = users.get(username)
    if user and user.password == password:
        print(f"User {username} logged in.")
        return user
    print("Invalid credentials.")
    return None

# Example registration and login
register_user("andi", "password123", is_seller=True)
login_user("andi", "password123")

In [ ]:
users = {}

def register_user(username: str, password: str):
    if username in users:
        print("Username already exists.")
        return None
    user = User(username, password)
    users[username] = user
    print(f"User {username} registered.")
    return user

def login_user(username: str, password: str):
    user = users.get(username)
    if user and user.password == password:
        print(f"User {username} logged in.")
        return user
    print("Invalid credentials.")
    return None

# Example registration and login
register_user("alice", "password123")
login_user("alice", "password123")

## 6. Seller Store & Product Management
Functions for sellers to create stores, add/edit/delete products, and manage payment methods.

In [ ]:
def create_store(seller: Seller, name: str, description: str):
    store_id = max(stores.keys(), default=0) + 1
    store = Store(store_id, seller, name, description)
    seller.stores.append(store)
    stores[store_id] = store
    print(f"Store '{name}' created for seller {seller.username}.")
    return store

def add_product(store: Store, name: str, category: str, price_idr: int, description: str, image_url: str = None):
    product_id = max([p.id for p in store.products], default=0) + 1
    product = Product(product_id, name, category, price_idr, description, image_url)
    store.products.append(product)
    print(f"Product '{name}' added to store '{store.name}'.")
    return product

def remove_product(store: Store, product_id: int):
    before = len(store.products)
    store.products = [p for p in store.products if p.id != product_id]
    after = len(store.products)
    if before == after:
        print("Product not found.")
    else:
        print(f"Product {product_id} removed from store '{store.name}'.")

def set_payment_methods(store: Store, methods: list, qris_image_url: str = None):
    store.payment_methods = methods
    store.qris_image_url = qris_image_url
    print(f"Payment methods updated for store '{store.name}'.")

# Example usage
andi = users["andi"]
my_store = create_store(andi, "Toko Andi 2", "Toko kebutuhan rumah tangga")
add_product(my_store, "Sabun Cuci Piring", "Kebutuhan Rumah", 12000, "Sabun cair 800ml", None)
set_payment_methods(my_store, ["Cash on Delivery", "Bank Transfer"], None)
store_products_df(my_store)

## 7. Order Placement and Payment Simulation
Simulate buyers placing orders, selecting payment methods, and sellers receiving order details.

In [ ]:
import uuid
from datetime import datetime

orders = {}

class Pesanan:
    def __init__(self, order_id, pembeli, toko, items, metode_pembayaran, bukti_pembayaran = None, alamat_pengiriman = None):
        self.order_id = order_id
        self.pembeli = pembeli
        self.toko = toko
        self.items = items  # List of (Produk, quantity)
        self.metode_pembayaran = metode_pembayaran
        self.bukti_pembayaran = bukti_pembayaran  # e.g., image URL for QRIS/bank transfer
        self.status = "Pesanan Baru"
        self.waktu_dibuat = datetime.now()
        self.waktu_status = [(self.status, self.waktu_dibuat)]
        self.alamat_pengiriman = alamat_pengiriman
    def total(self):
        return sum(produk.harga * qty for produk, qty in self.items)

def buat_pesanan(pembeli, toko, keranjang, metode_pembayaran, bukti_pembayaran = None, alamat_pengiriman = None):
    order_id = str(uuid.uuid4())
    pesanan = Pesanan(order_id, pembeli, toko, keranjang, metode_pembayaran, bukti_pembayaran, alamat_pengiriman)
    orders[order_id] = pesanan
    print(f"Pesanan {order_id} dibuat oleh {pembeli.username} di toko '{toko.nama}'. Total: Rp{pesanan.total():,}")
    print(f"Metode pembayaran: {metode_pembayaran}")
    if bukti_pembayaran:
        print(f"Bukti pembayaran diunggah: {bukti_pembayaran}")
    return pesanan

# Contoh: Pembeli guest memasukkan alamat saat checkout
guest = Pembeli("guest", "", nama="Tamu", alamat=None)
alamat_guest = "Jl. Melati No. 123, Perumahan Mawar"
keranjang_guest = []  # List of (Produk, qty)
# pesanan_guest = buat_pesanan(guest, toko, keranjang_guest, "COD", alamat_pengiriman=alamat_guest)

## 8. Order Status Management
Simulate updating and tracking order status for both buyer and seller (e.g., New Order, In Progress, Out for Delivery, Completed, Cancelled).

In [ ]:
STATUS_PESANAN = [
    "Pesanan Baru",
    "Diproses",
    "Dalam Pengiriman",
    "Selesai",
    "Dibatalkan"
 ]

def update_status_pesanan(pesanan, status_baru):
    if status_baru not in STATUS_PESANAN:
        print(f"Status tidak valid: {status_baru}")
        return
    status_lama = pesanan.status
    pesanan.status = status_baru
    waktu = datetime.now()
    pesanan.waktu_status.append((status_baru, waktu))
    print(f"Status pesanan {pesanan.order_id} diperbarui: {status_lama} → {status_baru}")

def get_status_pesanan(pesanan):
    print(f"Status pesanan {pesanan.order_id}: {pesanan.status}")
    return pesanan.status

# Contoh: Penjual memperbarui status pesanan
update_status_pesanan(order, "Diproses")
get_status_pesanan(order)
update_status_pesanan(order, "Dalam Pengiriman")
get_status_pesanan(order)
update_status_pesanan(order, "Selesai")
get_status_pesanan(order)

## 9. Notification Simulation
Simulate notifications for buyers and sellers when order status changes or new orders are placed.

In [ ]:
notifikasi = {"pembeli": {}, "penjual": {}}

def kirim_notifikasi(pengguna, pesan, peran):
    if pengguna.username not in notifikasi[peran]:
        notifikasi[peran][pengguna.username] = []
    notifikasi[peran][pengguna.username].append(pesan)
    print(f"[Notifikasi untuk {peran} '{pengguna.username}']: {pesan}")

def notifikasi_pesanan_baru(pesanan):
    penjual = pesanan.toko.penjual
    msg = f"Pesanan baru: {pesanan.order_id} dari {pesanan.pembeli.username} (Total: Rp{pesanan.total():,})"
    kirim_notifikasi(penjual, msg, "penjual")

def notifikasi_status_pesanan(pesanan):
    pembeli = pesanan.pembeli
    msg = f"Status pesanan {pesanan.order_id} diperbarui menjadi '{pesanan.status}'"
    kirim_notifikasi(pembeli, msg, "pembeli")
    penjual = pesanan.toko.penjual
    kirim_notifikasi(penjual, msg, "penjual")

# Contoh: Simulasi notifikasi
notifikasi_pesanan_baru(order)
update_status_pesanan(order, "Diproses")
notifikasi_status_pesanan(order)
update_status_pesanan(order, "Dalam Pengiriman")
notifikasi_status_pesanan(order)
update_status_pesanan(order, "Selesai")
notifikasi_status_pesanan(order)

# Lihat notifikasi
print("\nNotifikasi pembeli:", notifikasi["pembeli"][order.pembeli.username])
print("\nNotifikasi penjual:", notifikasi["penjual"][order.toko.penjual.username])

In [ ]:
# Struktur ulasan dan rating produk/toko
class Ulasan:
    def __init__(self, reviewer, rating: int, komentar: str, waktu=None):
        self.reviewer = reviewer  # username atau nama
        self.rating = rating  # 1-5
        self.komentar = komentar
        self.waktu = waktu or datetime.now()

# Contoh menambahkan ulasan ke produk/toko
def tambah_ulasan_produk(produk, reviewer, rating, komentar):
    if not hasattr(produk, 'ulasan_list'):
        produk.ulasan_list = []
    produk.ulasan_list.append(Ulasan(reviewer, rating, komentar))

def tambah_ulasan_toko(toko, reviewer, rating, komentar):
    if not hasattr(toko, 'ulasan_list'):
        toko.ulasan_list = []
    toko.ulasan_list.append(Ulasan(reviewer, rating, komentar))

# Contoh penggunaan:
tambah_ulasan_produk(order.items[0][0], order.pembeli.username, 5, "Produk bagus dan pengiriman cepat!")
tambah_ulasan_toko(order.toko, order.pembeli.username, 5, "Toko responsif dan ramah.")